In [1]:
# carwala.com newly lanched cars

In [2]:
# import required pacakges for web scrapping with beautifulsoap
from urllib.request import urlopen as uRequest
from urllib.error import HTTPError
from urllib.error import URLError
import pandas as pd
from bs4 import BeautifulSoup as soap

In [5]:
def requestURL(url):
        try:
            connectURL = uRequest(url)
            readURL = connectURL.read()
            connectURL.close()
        except HTTPError as e:
            print("HTTP Error, URL is not responding with exception: "+ str(e))
            return None
        except URLError as e:
            print("URL Error, URL is not currect with exception: "+ str(e))
            return None
        else:
            print("URL request is successful.")
        return readURL
    

carWalaURL = "https://www.carwale.com/new-car-launches/"
requestCarWala = requestURL(carWalaURL)
soapObjCarWala = soap(requestCarWala, "html.parser")
print(soapObjCarWala.body.div)

URL request is successful.
<div class="blackOut-window" id="globalPopupBlackOut"></div>


In [34]:
detailsDiv = soapObjCarWala.findAll("div",{"class":"grid-7 omega"})
for child in detailsDiv:
        print("Name: "+ str(child.find("strong", {"class": "text-unbold"}).get_text()))
        print("price: "+ str(child.find("div",{"class": "font20 margin-top15"}).get_text()))
        print("price based on: "+ str(child.find("div", {"class": "margin-bottom5"}).text))
        print("Lanched on: "+ str(child.find("div", {"class": "margin-bottom15 font14"}).get_text().split(":")[1].strip()))
        print("rating: "+ str(child.find("div", {"class": "margin-bottom10"}).a.get_text().strip()))

Name: Volkswagen Tiguan AllSpace
price:   ₹ 33.13 Lakhs  onwards 
price based on: Avg. Ex-Showroom price
Lanched on: 06-Mar-2020
rating: 2 User Reviews
Name: BMW X1
price:   ₹ 35.9 Lakhs  onwards 
price based on: Avg. Ex-Showroom price
Lanched on: 05-Mar-2020
rating: Be the first one to write a review
Name: Mercedes-Benz GLC Coupe
price:   ₹ 62.7 Lakhs  onwards 
price based on: Avg. Ex-Showroom price
Lanched on: 03-Mar-2020
rating: 1 User Review
Name: Toyota Vellfire
price:   ₹ 79.5 Lakhs  onwards 
price based on: Avg. Ex-Showroom price
Lanched on: 26-Feb-2020
rating: Be the first one to write a review
Name: Land Rover Defender
price:   ₹ 69.99 Lakhs  onwards 
price based on: Avg. Ex-Showroom price
Lanched on: 26-Feb-2020
rating: Be the first one to write a review
Name: Maruti Suzuki Vitara Brezza
price:   ₹ 7.34 Lakhs  onwards 
price based on: Avg. Ex-Showroom price
Lanched on: 24-Feb-2020
rating: 37 User Reviews
Name: Maruti Suzuki Ignis
price:   ₹ 4.89 Lakhs  onwards 
price based on

In [45]:
print(soapObjCarWala.find("div", {"id": "rptContent"}).find("span", {"id": "divLastNav"}).span.text)
print(soapObjCarWala.find("div", {"id": "rptContent"}).find("span", {"id": "divLastNav"}).a['href'])

Next
/new-car-launches/page/2/


In [113]:
# Now take all the Newly lanched car details and update in Excel
def carWalaData(url, baseURL=None):
        try:
            global listObjectForCarDict
            requestURL = uRequest(url)
            readURL = requestURL.read()
            requestURL.close()
        except HTTPError as e:
            print("Exception while requesting: "+ str(e))
            return None
        except URLError as e:
            print("Exception while requesting: "+ str(e))
            return None
        else:
            print("URL Request is sucessful and retrived data.")
        try:
            soapObject = soap(readURL, "html.parser")
            print("soap Object is updated sucessfully.")
            # list of all newly lanched cars from carwala
            collectionOfCarData = soapObject.findAll("div",{"class":"grid-7 omega"})
            for eachCollection in collectionOfCarData:
                    carsDetails = {}
                    try:
                        carName =  eachCollection.find("strong", {"class": "text-unbold"}).get_text()
                    except AttributeError as e:
                        carName = None
                        print("Exception carDetails: "+ str(e))
                    carsDetails['carName'] = carName
                    try:
                        carPrice =  eachCollection.find("div",{"class": "font20 margin-top15"}).get_text()
                    except AttributeError as e:
                        carPrice = None
                        print("Exception carPrice: "+ str(e))
                    carsDetails['carPrice'] = carPrice
                    try:
                        carPriceBasedOn =  eachCollection.find("div", {"class": "margin-bottom5"}).text
                    except AttributeError as e:
                        carPriceBasedOn = None
                        print("Exception BasedOn: "+ str(e))
                    carsDetails['carPriceBasedOn'] = carPriceBasedOn
                    try:
                        lanchDate = eachCollection.find("div", {"class": "margin-bottom15 font14"}).get_text().split(":")[1].strip()
                    except AttributeError as e:
                        lanchDate = None
                        print("Exception  lanchDate: "+ str(e))
                    carsDetails['lanchDate'] = lanchDate
                    try:
                        carRating = eachCollection.find("div", {"class": "margin-bottom10"}).a.get_text().strip()
                    except AttributeError as e:
                        carRating = None
                        print("Exception on carRating: "+ str(e))
                    carsDetails['carRating'] = carRating
                    listObjectForCarDict.append(carsDetails)
            tempFrame = pd.DataFrame(listObjectForCarDict)
        except Exception as exc:
            print("Exception out: "+ str(exc))
        
        try:
            try:
                link = soapObject.find("div", {"id": "rptContent"}).find("span", {"id": "divLastNav"}).a['href']
            except AttributeError as exc:
                link = None
                print("AttributeError.")
            print(link)    
            if link is not None:
                nextURL = baseURL+soapObject.find("div", {"id": "rptContent"}).find("span", {"id": "divLastNav"}).a['href']
                carWalaFrame = carWalaData(nextURL, baseURL)
                return carWalaFrame
            else:
                tempFrame = tempFrame.append(tempFrame)
                return tempFrame
        except Exception as exc:
            print("Exception outside: "+ str(exc))
            return tempFrame
listObjectForCarDict = []
baseURL = "https://www.carwale.com"                 
url = "https://www.carwale.com/new-car-launches/"
carDataFrame = pd.DataFrame()
carDataFrame = carWalaData(url, baseURL)
carDataFrame.to_csv("C:\\Users\\pvalluru\\Desktop\\Python_practice\\Pandas\\dataset\\carwala_newly_lanched_cars.csv", index= False)
carDataFrame

URL Request is sucessful and retrived data.
soap Object is updated sucessfully.
/new-car-launches/page/2/
URL Request is sucessful and retrived data.
soap Object is updated sucessfully.
/new-car-launches/page/3/
URL Request is sucessful and retrived data.
soap Object is updated sucessfully.
/new-car-launches/page/4/
URL Request is sucessful and retrived data.
soap Object is updated sucessfully.
/new-car-launches/page/5/
URL Request is sucessful and retrived data.
soap Object is updated sucessfully.
/new-car-launches/page/6/
URL Request is sucessful and retrived data.
soap Object is updated sucessfully.
Exception outside: 'NoneType' object is not subscriptable


,carName,carPrice,carPriceBasedOn,lanchDate,carRating
0,Volkswagen Tiguan AllSpace,₹ 33.13 Lakhs onwards,Avg. Ex-Showroom price,06-Mar-2020,2 User Reviews
1,BMW X1,₹ 35.9 Lakhs onwards,Avg. Ex-Showroom price,05-Mar-2020,Be the first one to write a review
2,Mercedes-Benz GLC Coupe,₹ 62.7 Lakhs onwards,Avg. Ex-Showroom price,03-Mar-2020,1 User Review
3,Toyota Vellfire,₹ 79.5 Lakhs onwards,Avg. Ex-Showroom price,26-Feb-2020,Be the first one to write a review
4,Land Rover Defender,₹ 69.99 Lakhs onwards,Avg. Ex-Showroom price,26-Feb-2020,Be the first one to write a review
5,Maruti Suzuki Vitara Brezza,₹ 7.34 Lakhs onwards,Avg. Ex-Showroom price,24-Feb-2020,38 User Reviews
6,Maruti Suzuki Ignis,₹ 4.89 Lakhs onwards,Avg. Ex-Showroom price,18-Feb-2020,6 User Reviews
7,Land Rover Discovery Sport,₹ 57.06 Lakhs onwards,Avg. Ex-Showroom price,13-Feb-2020,Be the first one to write a review
8,Kia Carnival,₹ 24.95 Lakhs onwards,Avg. Ex-Showroom price,05-Feb-2020,11 User Reviews
9,Mercedes-Benz AMG GT 4-Door Coupe,₹ 2.42 Crores onwards,Avg. Ex-Showroom price,05-Feb-2020,Be the first one to write a review
